# FINDATA 03 - 종목별 과거 시세

#### 이승준 http://fb.com/plusjune

# 네이버 파이낸스 크롤링

* http://finance.naver.com
* http://finance.naver.com/item/sise_day.nhn?code=105560&page=1

# 대표적인 두가지 방법
1. pd.read_html()
1. BeautifulSoup, requests

# DataFrame으로 읽기

In [1]:
import pandas as pd

url = 'http://finance.naver.com/item/sise_day.nhn?code=105560&page=1'
dfs = pd.read_html(url)

In [2]:
len(dfs)

2

In [3]:
df_price = dfs[0]
df_price

,0,1,2,3,4,5,6
0,날짜,종가,전일비,시가,고가,저가,거래량
1,2016.05.04,33500,450,33650,33900,33150,987759
2,2016.05.03,33950,150,34100,34550,33750,962139
3,2016.05.02,34100,700,34400,34750,34000,900767
4,2016.04.29,34800,450,35250,35250,34350,1009891
5,2016.04.28,35250,400,36050,36050,34950,1059990
6,2016.04.27,35650,50,35300,36250,35200,1207171
7,2016.04.26,35700,800,35900,36200,35500,1101442
8,2016.04.25,36500,600,35900,36600,35550,1110006
9,2016.04.22,35900,500,35550,36150,35350,1645260


In [4]:
df_price.dtypes

0    object
1    object
2    object
3    object
4    object
5    object
6    object
dtype: object

In [5]:
df_price.columns = ['date', 'close', 'change', 'open', 'high', 'low', 'volume']
df_price = df_price[1:] 
df_price = df_price.replace('\.', '-', regex=True)
df_price['date'] = pd.to_datetime(df_price['date'])

int_cols = ['close', 'change', 'open', 'high', 'low', 'volume']
df_price[int_cols] = df_price[int_cols].astype('int')
df_price

,date,close,change,open,high,low,volume
1,2016-05-04,33500,450,33650,33900,33150,987759
2,2016-05-03,33950,150,34100,34550,33750,962139
3,2016-05-02,34100,700,34400,34750,34000,900767
4,2016-04-29,34800,450,35250,35250,34350,1009891
5,2016-04-28,35250,400,36050,36050,34950,1059990
6,2016-04-27,35650,50,35300,36250,35200,1207171
7,2016-04-26,35700,800,35900,36200,35500,1101442
8,2016-04-25,36500,600,35900,36600,35550,1110006
9,2016-04-22,35900,500,35550,36150,35350,1645260
10,2016-04-21,35400,600,35000,35450,34850,966479


In [6]:
df_price.dtypes

date      datetime64[ns]
close              int64
change             int64
open               int64
high               int64
low                int64
volume             int64
dtype: object

# DB 테이블로 저장

In [7]:
import pandas as pd
import sqlite3

# KB금융(105560)

url = 'http://finance.naver.com/item/sise_day.nhn?code=105560&page=1'
dfs = pd.read_html(url)

In [8]:
conn = sqlite3.connect('findata.db')

df_price.to_sql('test_table', conn, index=False, if_exists='replace')
conn.close()

In [9]:
conn = sqlite3.connect('findata.db')

df_price = pd.read_sql('select * from test_table', conn, parse_dates=['date'])
df_price.head()

,date,close,change,open,high,low,volume
0,2016-05-04,33500,450,33650,33900,33150,987759
1,2016-05-03,33950,150,34100,34550,33750,962139
2,2016-05-02,34100,700,34400,34750,34000,900767
3,2016-04-29,34800,450,35250,35250,34350,1009891
4,2016-04-28,35250,400,36050,36050,34950,1059990


In [10]:
df_price.dtypes

date      datetime64[ns]
close              int64
change             int64
open               int64
high               int64
low                int64
volume             int64
dtype: object

In [11]:
conn.close()

# 페이지 네비게이션
http://finance.naver.com/item/sise_day.nhn?code=105560&page=1
1. 마지막 페이지 숫자 알아내기
1. requests, BeautifulSoup 사용

In [12]:
from bs4 import BeautifulSoup
import requests

In [13]:
url = 'http://finance.naver.com/item/sise_day.nhn?code=105560&page=1'
r = requests.get(url)
soup = BeautifulSoup(r.text, "lxml")

In [14]:
td = soup.find('td', attrs={'class':'pgRR'})
td

<td class="pgRR">
<a href="/item/sise_day.nhn?code=105560&amp;page=188">맨뒤
				<img alt="" border="0" height="5" src="http://static.naver.net/n/cmn/bu_pgarRR.gif" width="8"/>
</a>
</td>

In [15]:
td = soup.find('td', attrs={'class':'pgRR'})
npage = int(td.a['href'].split('page=')[1])
npage

188

In [16]:
from bs4 import BeautifulSoup
import requests

def get_last_page_num(code):
    url = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=1' % (code)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    td = soup.find('td', attrs={'class':'pgRR'})
    npage = td.a['href'].split('page=')[1]
    return int(npage)

In [17]:
get_last_page_num('005930') # 삼성전자

502

In [18]:
get_last_page_num('105560') # KB금융

188

# 페이지 전체 네비게이션

In [19]:
url_tmpl = 'http://finance.naver.com/item/sise_day.nhn?code=105560&page=%d'

npages = get_last_page_num('105560')
for p in range(1, npages + 1):
    url = url_tmpl % p


In [20]:
df_price['code'] = '105560'
df_price

,date,close,change,open,high,low,volume,code
0,2016-05-04,33500,450,33650,33900,33150,987759,105560
1,2016-05-03,33950,150,34100,34550,33750,962139,105560
2,2016-05-02,34100,700,34400,34750,34000,900767,105560
3,2016-04-29,34800,450,35250,35250,34350,1009891,105560
4,2016-04-28,35250,400,36050,36050,34950,1059990,105560
5,2016-04-27,35650,50,35300,36250,35200,1207171,105560
6,2016-04-26,35700,800,35900,36200,35500,1101442,105560
7,2016-04-25,36500,600,35900,36600,35550,1110006,105560
8,2016-04-22,35900,500,35550,36150,35350,1645260,105560
9,2016-04-21,35400,600,35000,35450,34850,966479,105560


# 전종목 과거 가격 데이터저장

In [ ]:
# stock_price 테이블 생성
conn = sqlite3.connect('findata.db')

sql = '''
CREATE TABLE "stock_price" (    
  "date" TIMESTAMP,
  "code" TEXT,
  "close" INTEGER,
  "change" INTEGER,
  "open" INTEGER,
  "high" INTEGER, 
  "low" INTEGER,  
  "volume" INTEGER
);
'''
c = conn.cursor()
c.execute(sql)
conn.close()

In [ ]:
# stock_price 테이블 삭제
conn = sqlite3.connect('findata.db')

sql = 'DROP TABLE stock_price'
c = conn.cursor()
c.execute(sql)
conn.close()

In [22]:
import sqlite3
conn = sqlite3.connect('findata.db')

code = '105560'

url_tmpl = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=%d'

npages = get_last_page_num(code)

for p in range(npages, 0, -1):
    url = url_tmpl % (code, p)
    dfs = pd.read_html(url)
    df_price = dfs[0]
    df_price.columns = ['date', 'close', 'change', 'open', 'high', 'low', 'volume']
    df_price = df_price[1:] 
    df_price = df_price.replace('\.', '-', regex=True)
    df_price['date'] = pd.to_datetime(df_price['date'])
    int_cols = ['close', 'change', 'open', 'high', 'low', 'volume']
    df_price[int_cols] = df_price[int_cols].astype('int')
    df_price['code'] = code
    df_price.to_sql('stock_price', conn, if_exists='append', index=False)
    print('%d,' % p, end='')

188,187,186,185,184,183,182,181,180,179,178,177,176,175,174,173,172,171,170,169,168,167,166,165,164,163,162,161,160,159,158,157,156,155,154,153,152,151,150,149,148,147,146,145,144,143,142,141,140,139,138,137,136,135,134,133,132,131,130,129,128,127,126,125,124,123,122,121,120,119,118,117,116,115,114,113,112,111,110,109,108,107,106,105,104,103,102,101,100,99,98,97,96,95,94,93,92,91,90,89,88,87,86,85,84,83,82,81,80,79,78,77,76,75,74,73,72,71,70,69,68,67,66,65,64,63,62,61,60,59,58,57,56,55,54,53,52,51,50,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,

In [23]:
import sqlite3
conn = sqlite3.connect('findata.db')

def stock_code_price_to_sql(code):
    df_max = pd.read_sql('SELECT * FROM stock_price where code="%s" ORDER BY date DESC LIMIT 1', conn)
    
    url_tmpl = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=%d'
    npages = get_last_page_num(code)
    for p in range(npages, 0, -1):
        url = url_tmpl % (code, p)
        dfs = pd.read_html(url)
        df_price = dfs[0]
        df_price.columns = ['date', 'close', 'change', 'open', 'high', 'low', 'volume']
        df_price = df_price[1:] 
        df_price = df_price.replace('\.', '-', regex=True)
        df_price['date'] = pd.to_datetime(df_price['date'])
        df_price[int_cols] = df_price[int_cols].astype('int', raise_on_error=False)
        df_price['code'] = code
        df_price.to_sql('stock_price', conn, if_exists='append', index=False)
        print('%d,' % p, end='')
    conn.close()
    print()

In [ ]:
stock_code_price_to_sql('105560')

188,187,186,185,184,183,182,181,180,179,178,177,176,175,174,173,172,171,170,169,168,167,166,165,164,163,162,161,160,

In [ ]:
conn.close()

# 데이터 확인

In [245]:
%matplotlib inline

import sqlite3
conn = sqlite3.connect('findata.db')

df_price = pd.read_sql('SELECT * FROM stock_price limit 30', conn)
df_price.head()

# 테이블  읽기
df_price = pd.read_sql(
    'SELECT * '
    'FROM stock_price '
    'WHERE code="079550" and date BETWEEN "2015-01-01" AND "2016-04-30"', 
    conn,
    index_col='date',)

df_price['close'].plot(figsize=(14,4), grid=True)

conn.close()

,date,code,close,change,open,high,low,volume
0,2016-05-04 00:00:00,105560,33500,450,33650,33900,33150,987759
1,2016-05-03 00:00:00,105560,33950,150,34100,34550,33750,962139
2,2016-05-02 00:00:00,105560,34100,700,34400,34750,34000,900767
3,2016-04-29 00:00:00,105560,34800,450,35250,35250,34350,1009891
4,2016-04-28 00:00:00,105560,35250,400,36050,36050,34950,1059990


# 전체 다운로드
(주의: 전체 다운로드를 위해서는 4~5일 정도 소요됩니다)

In [ ]:
# 마스터(코드+종목명) 읽기
conn = sqlite3.connect('findata.db')
df_master = pd.read_sql('SELECT * FROM stock_master', conn)
df_master.head()

# 모든 종목에 대해 데이터 다운로드
for inx, row in df_master.iterrows():
    stock_code_price_to_sql(row['code'])
    
conn.close()

# 코드 전체 (stock_price 테이블 만드는)
* 다음 코드를 .py 파일로 저장한 뒤 실행하면 됩니다. 
* 만일 실행 중간에 중단된 경우, 다시 실행하면 마지막에 다운로드하지 못한 종목 부터 다시 시작 합니다.
* '2016-05-04 00:00:00' 은 마지막 영업일로 지정해 주십시오.

In [ ]:
# stock_price_to_sql.py

import sqlite3
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests

def get_last_page_num(code):
    url = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=1' % (code)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    td = soup.find('td', attrs={'class':'pgRR'})
    npage = td.a['href'].split('page=')[1]
    return int(npage)
    
def stock_code_price_to_sql(code):
    conn = sqlite3.connect('findata.db')
        
    df_max = pd.read_sql('SELECT * FROM stock_price where code="%s" ORDER BY date DESC LIMIT 1', conn)
    
    url_tmpl = 'http://finance.naver.com/item/sise_day.nhn?code=%s&page=%d'
    npages = get_last_page_num(code)
    for p in range(npages, 0, -1):
        url = url_tmpl % (code, p)
        dfs = pd.read_html(url)
        df_price = dfs[0]
        df_price.columns = ['date', 'close', 'change', 'open', 'high', 'low', 'volume']
        df_price = df_price[1:] 
        df_price = df_price.replace('\.', '-', regex=True)
        df_price['date'] = pd.to_datetime(df_price['date'])
        int_cols = ['close', 'change', 'open', 'high', 'low', 'volume']
        df_price[int_cols] = df_price[int_cols].astype('int', raise_on_error=False)
        df_price['code'] = code
        df_price.to_sql('stock_price', conn, if_exists='append', index=False)
        print('%d,' % p, end='', flush=True)
    conn.close()
    print()
    
if __name__ == "__main__":
    conn = sqlite3.connect('findata.db')
    df_master = pd.read_sql('SELECT * FROM stock_master', conn)
    df_master.head()

    for inx, row in df_master.iterrows():
        conn = sqlite3.connect('findata.db')
        df_max = pd.read_sql('SELECT MAX (date) AS "maxdate" FROM stock_price WHERE code="%s"' % row['code'], conn)
        conn.close()
    
        if df_max['maxdate'][0] and (df_max['maxdate'][0] >= '2016-05-04 00:00:00'):
            print(row['code'], row['name'], 'UPDATED')
            continue
        print(row['code'], row['name'])
        stock_code_price_to_sql(row['code'])
    